In [1]:
import numpy as np
import pandas as pd
from pandas import pivot_table
import matplotlib.pyplot as plt
import seaborn as sns
import json
from colour import Color

In [2]:
borig = pd.read_csv('2018clean3.csv', header=0, index_col=0, dtype={'MainSic': object})

print(borig.shape)
borig.head(2)

(10772, 29)


,EmployerName,Address,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,...,ResponsiblePerson,EmployerSize,CurrentName,SubmittedAfterTheDeadline,DueDate,DateSubmitted,Post,Val,MainSic,MainSection
0,"BRYANSTON SCHOOL,INCORPORATED","Bryanston House,\r\r\r\r\nBlandford,\r\r\r\r\n...",00226143,85310,19.6,32.5,NaN,NaN,0.0,0.0,...,Michael McGovern (Chief Operating Officer),500 to 999,"""Bryanston School"",Incorporated",False,05/04/2019 00:00:00,03/04/2019 13:43:34,DT11 0PX,True,85310,Education
1,"RED BAND CHEMICAL COMPANY, LIMITED","19 Smith's Place,\r\r\r\r\nLeith Walk,\r\r\r\r...",SC016876,47730,7.7,-4.2,23.7,-75.2,22.2,82.0,...,Philip Galt (Managing Director),250 to 499,"""RED BAND"" CHEMICAL COMPANY, LIMITED",False,05/04/2019 00:00:00,01/04/2019 12:37:21,EH6 8NU,True,47730,Wholesale and retail trade; repair of motor ve...


In [3]:
addnuts = pd.read_csv('pc2018_uk_NUTS-2016_v2.0.csv', sep=';')

print(addnuts.shape)
addnuts.head()

(1759911, 2)


,NUTS3,CODE
0,'UKM73','EH10 7DP'
1,'UKM73','EH10 7DX'
2,'UKM73','EH10 7DY'
3,'UKM73','EH10 7DZ'
4,'UKM73','EH10 7EA'


In [4]:
def remove_quotes(x):
    return str(x)[1:-1]

In [5]:
addnuts['NUTS3'] = addnuts['NUTS3'].apply(remove_quotes)
addnuts['CODE'] = addnuts['CODE'].apply(remove_quotes)

addnuts.head(2)

,NUTS3,CODE
0,UKM73,EH10 7DP
1,UKM73,EH10 7DX


In [6]:
addnuts = addnuts[['CODE', 'NUTS3']]
addnuts.rename(columns={"CODE": "Post", "NUTS3": "Nuts3"}, inplace=True)

addnuts.head(2)

,Post,Nuts3
0,EH10 7DP,UKM73
1,EH10 7DX,UKM73


In [7]:
len(addnuts['Nuts3'].unique())

179

In [8]:
borig = pd.merge(borig, addnuts, on='Post')

borig.sample(2)

,EmployerName,Address,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,...,EmployerSize,CurrentName,SubmittedAfterTheDeadline,DueDate,DateSubmitted,Post,Val,MainSic,MainSection,Nuts3
5121,HELENA PARTNERSHIPS LIMITED,"Helena Central,\r\r\r\r\n4 Corporation Street,...",04141767,"41100,\r\r\r\r\n68201,\r\r\r\r\n81300",1.4,0.4,-3.6,NaN,7.6,1.7,...,250 to 499,HELENA PARTNERSHIPS LIMITED,False,05/04/2019 00:00:00,15/01/2019 11:10:19,WA9 1LD,True,41100,Construction,UKD71
3403,"DEPARTMENT FOR DIGITAL, CULTURE, MEDIA AND SPORT","100 Parliament Street,\r\r\r\r\nLondon,\r\r\r\...",NaN,"1,\r\r\r\r\n84110",7.5,22.9,25.4,9.1,38.2,45.8,...,500 to 999,"Department for Digital, Culture, Media and Sport",False,31/03/2019 00:00:00,19/12/2018 14:05:59,SW1A 2BQ,True,84110,Public administration and defence; compulsory ...,UKI32


In [9]:
n_piv = pivot_table(borig, values='DiffMeanHourlyPercent',
                     index=['Nuts3'], aggfunc=np.mean)


n_piv.rename(columns={'DiffMeanHourlyPercent':'Diff_avg'}, inplace=True)
n_piv.sort_values(by=['Diff_avg'], ascending=False, inplace=True)
n_piv['Diff_avg'] = round(n_piv['Diff_avg'], 2)
n_piv.reset_index(inplace=True)

n_piv.head(5)

,Nuts3,Diff_avg
0,UKN14,25.55
1,UKM61,24.30
2,UKI62,22.57
3,UKH32,21.74
4,UKI42,20.71


In [10]:
nuts_name = pd.read_csv('NUTS_Level_3_January_2018_Ultra_Generalised_Clipped_Boundaries_in_the_United_Kingdom.csv')
print(nuts_name.shape)
nuts_name.head(2)

(179, 9)


,objectid,nuts318cd,nuts318nm,bng_e,bng_n,long,lat,st_areashape,st_lengthshape
0,1,UKC11,Hartlepool and Stockton-on-Tees,444952,522016,-1.30587,54.591351,3.014553e+08,108085.925509
1,2,UKC12,South Teesside,461718,519597,-1.04695,54.567848,3.027700e+08,86922.520900


In [11]:
nuts_name = nuts_name[['nuts318cd','nuts318nm','lat','long']]
nuts_name.rename(columns={'nuts318cd':'Nuts3','nuts318nm':'Name','long':'lon'}, inplace=True)

nuts_name.head()

,Nuts3,Name,lat,lon
0,UKC11,Hartlepool and Stockton-on-Tees,54.591351,-1.30587
1,UKC12,South Teesside,54.567848,-1.04695
2,UKC13,Darlington,54.535351,-1.56835
3,UKC14,Durham CC,54.685131,-1.84050
4,UKC21,Northumberland,55.300369,-2.07521


In [12]:
n_piv = pd.merge(n_piv, nuts_name, on='Nuts3')

n_piv.tail(7)

,Nuts3,Diff_avg,Name,lat,lon
165,UKM81,6.37,"East Dunbartonshire, West Dunbartonshire and H...",56.107639,-4.71665
166,UKI71,5.34,Barnet,51.611080,-0.21821
167,UKM94,4.96,South Ayrshire,55.230068,-4.72901
168,UKL14,3.03,South West Wales,52.085041,-4.18274
169,UKN11,1.13,Mid Ulster,54.552650,-6.88801
170,UKN07,-1.13,"Armagh City, Banbridge and Craigavon",54.386600,-6.43360
171,UKN15,-5.60,Mid and East Antrim,54.864559,-6.14545


In [13]:
# exclude Wales, Scotland and N Ireland due to no geojson data
print(f'n_piv length with Wales, Scotland and N Ireland: {len(n_piv)}')

n_piv = n_piv[n_piv['Nuts3'].str.contains('UKL') == False]
n_piv = n_piv[n_piv['Nuts3'].str.contains('UKM') == False]
n_piv = n_piv[n_piv['Nuts3'].str.contains('UKN') == False]
n_piv.reset_index(inplace=True)

print(f'n_piv length without Wales, Scotland and N Ireland: {len(n_piv)}')

n_piv length with Wales, Scotland and N Ireland: 172
n_piv length without Wales, Scotland and N Ireland: 133


In [14]:
n_piv.head(2)

,index,Nuts3,Diff_avg,Name,lat,lon
0,2,UKI62,22.57,Croydon,51.365978,-0.077610
1,3,UKH32,21.74,Thurrock,51.509941,0.334902


In [15]:
# n_piv.to_csv('2018nuts3avg.csv')

## Company Min Max

In [16]:
bedit = borig.copy()

bedit.shape

(10590, 30)

In [17]:
bedit = bedit[bedit['Nuts3'].str.contains('UKL') == False]
bedit = bedit[bedit['Nuts3'].str.contains('UKM') == False]
bedit = bedit[bedit['Nuts3'].str.contains('UKN') == False]

bedit.shape

(9754, 30)

In [18]:
bedit = bedit[['Nuts3', 'EmployerName', 'DiffMeanHourlyPercent']]
bedit.rename(columns={'DiffMeanHourlyPercent':'Diff_avg'}, inplace=True)
bedit.reset_index(drop=True, inplace=True)

bedit.head(2)

,Nuts3,EmployerName,Diff_avg
0,UKK22,"BRYANSTON SCHOOL,INCORPORATED",19.6
1,UKE42,123 EMPLOYEES LTD,30.0


In [19]:
print(bedit.Nuts3.value_counts()[:2])
print(bedit.Nuts3.value_counts()[-2:])

UKI31    735
UKI32    567
Name: Nuts3, dtype: int64
UKK42    9
UKJ34    7
Name: Nuts3, dtype: int64


In [20]:
bedit.head()

,Nuts3,EmployerName,Diff_avg
0,UKK22,"BRYANSTON SCHOOL,INCORPORATED",19.6
1,UKE42,123 EMPLOYEES LTD,30.0
2,UKJ25,1509 GROUP,13.4
3,UKK23,1610 LIMITED,-25.0
4,UKH12,1LIFE MANAGEMENT SOLUTIONS LIMITED,0.9


In [21]:
len(bedit.Nuts3.unique())

133

In [22]:
bmm = pd.DataFrame(columns=bedit.columns)

for i in bedit.Nuts3.unique():
    bsample = bedit[bedit.Nuts3 == i]
    
    bmm = bmm.append(bsample[bsample.Diff_avg == bsample.Diff_avg.max()].iloc[0,:], ignore_index=True)
    bmm = bmm.append(bsample[bsample.Diff_avg == bsample.Diff_avg.min()].iloc[0,:], ignore_index=True)
    
bmm.reset_index(drop=True, inplace=True)

print(bmm.shape)
bmm.head()

(266, 3)


,Nuts3,EmployerName,Diff_avg
0,UKK22,WIMBORNE ACADEMY TRUST,38.7
1,UKK22,CLIPPER CONTRACTING GROUP LTD,-8.6
2,UKE42,LEEDS UNITED FOOTBALL CLUB LIMITED,66.3
3,UKE42,NATIONWIDE WINDOW CLEANING LIMITED,-44.6
4,UKJ25,SHL GROUP LIMITED,51.6


In [23]:
# bmm.to_csv('2018nuts3minmax.csv')